# Forward ad backward passes

## Data

In [1]:
# Download the MNIST dataset
import gzip
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
from pathlib import Path
from urllib.request import urlretrieve

MNIST_URL = "https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true"

data_path = Path("data/mnist")
data_path.mkdir(exist_ok=True)
gz_path = data_path / "mnist.pkl.gz"

mpl.rcParams["image.cmap"] = "gray"

if not gz_path.exists():
    urlretrieve(MNIST_URL, gz_path)

# File contains a tuple of tuples for the x and y, train and validation data
# Images are 28x28
with gzip.open(gz_path, "rb") as file:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(file, encoding="latin-1")

# Put into tensors
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

# We want our data as floats so we can use MSE
x_train = x_train.float()
y_train = y_train.float()

## Foundations version
 
- consider a linear model straight line on a graph. Limiting, what if what we want to approximate is a curvy line.
- summing multiple rectified lines/models together lets us create any shape we wanted if we had enough lines
- works the same in N dimensions

In [2]:
# Lets define some constants that describe the data
n_examples, n_pixels = x_train.shape
possible_values = y_train.max() + 1

# How many nodes/activations/line thingys
n_hidden = 50

n_examples, n_pixels, possible_values

(50000, 784, tensor(10.))

In [3]:
import torch

# For a simple linear model
# 50000x784 (examples x pixels)  @ 784x10 (pixels x weights for each possible value)
# = 50000x10 (results for 50000 images)
#
# However we have 50 hidden layers so we want to to go into 784x50 and then 50000x10.

# Weights and bias for hidden layer
h_weights = torch.randn(n_pixels, n_hidden)  # 784x50
h_bias = torch.zeros(n_hidden)

# Output layer, for now we are just going to create 1 output rather than 10 to tell use the prediced number
# This will be a bit daft when we calculate the loss (as numbers further away from each other aren't more wrong)
# but makes some of the other calculations easier for now
o_weights = torch.randn(n_hidden, 1)  # Take outputs from hidden and smush into 1
o_bias = torch.zeros(1)

In [4]:
# Function to put x through a linear layer with specified bweights and bias
def lin(x, weights, bias):
    return x @ weights + bias

In [5]:
t = lin(x_train, h_weights, h_bias)
t.shape

torch.Size([50000, 50])

In [6]:
# Will need to put them relu
def relu(x):
    return x.clamp_min(0.0)

In [7]:
t = relu(t)
t

tensor([[ 0.0000,  0.0000,  9.2682,  ...,  1.3954,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  4.4059,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  9.7413,  ...,  9.1958,  6.1656,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ..., 10.5230,  0.0000,  3.1906],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  8.7968,  ...,  3.0232,  0.0000, 13.5904]])

In [8]:
# We can put this toghether in a baslic MLP
def model(in_data):
    h_out = lin(in_data, h_weights, h_bias)
    h_out = relu(h_out)

    o_out = lin(h_out, o_weights, o_bias)

    return o_out

In [9]:
# Well at least it tried
res = model(x_train)
res, res.shape

(tensor([[23.2929],
         [51.0174],
         [94.4659],
         ...,
         [90.0241],
         [88.1075],
         [83.3951]]),
 torch.Size([50000, 1]))

### MSE

In [10]:
# Because of the shape of our data and broadcasting rules we cant just do this
(res - y_train).shape

torch.Size([50000, 50000])

In [11]:
# As the broadcasting rules have broadcast 500000 into our 1
# We need to turn out 10000 vector into a 10000x1 matrix or turn our matrix into a vector
res[:, 0].shape

torch.Size([50000])

In [12]:
# .squeeze can do it too, it removes all unit dimensions
res.squeeze().shape

torch.Size([50000])

In [13]:
(res.squeeze() - y_train).shape

torch.Size([50000])

In [14]:
# So here is our mse
def mse(result, actual):
    return (result.squeeze() - actual).pow(2).mean()


mse(res, y_train)

tensor(6555.3013)

## Gradients and backward pass

We have `loss = L(f(x, weights), y)`. If we get the derivative of the loss with respect to a weight (`dloss/wi`) we would know how to change the weight to reduce the loss. We can then update each weight and go again. Thats SGD.

We have a whole bunch of weights and a whole bunch of inputs (pixels). And we also have multiple images to input. So we and up with a matrix of derivatives (the Jacobian) (a row for every input and a col for every output). We eventually want to end up with a single number for every input as our loss will need to be a single number.

In [15]:
# sympy could calulate derivatives symbolically for use
from sympy import symbols, diff

x, y = symbols("x y")
diff(x**2, x)

2*x

In [16]:
diff(3 * x**2 + 9, x)

6*x

Why is the derivative  of 3*x**2+9 = 6x?

The derivative of `a**b` with respect to `a` is `ba**b-1`, `da**b/da = ba**b-1`.

So we could rewrite as:

```
y = 3u + 9

where:
u = x**2
```

The derivative of a sum is the sum of the derivatives. The derivative of a constant is 0, as changes dont effect it. So

```
dy/du = 3 + 0
```

and

```
dy/dx = dy/du * du/dx  <-- this is the chain rule

dy/dx = 2x * 3 = 6x
```

We want to calculate the gradient of our MSE with respect to the input weights of the model. We cant do it symbolically so we will start at the end and use the chain rule to calculate the derivative. This is called backpropagation.

We can implement this manually.

In [17]:
def lin_grad(inp, out, weights, bias):
    # grad of the matrix multiplication with respect to input
    inp.g = out.g @ weights.t()

    weights.g = inp.T @ out.g
    # ^This is the same as:
    # weights.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)

    bias.g = out.g.sum(0)


def forward_and_backward(inp, actual):
    # forward pass (run the model)
    l1 = lin(inp, h_weights, h_bias)
    l2 = relu(l1)
    out = lin(l2, o_weights, o_bias)

    # MSE in 2 steps
    diff = out[:, 0] - actual
    loss = diff.pow(2).mean()

    # backward pass (calc the gradient), well store the gradients for each layer in g, working backwards
    out.g = 2.0 * diff[:, None] / inp.shape[0]  # deal with pow and mean

    # grad of l2, we need to know grad of out to do chain rule
    lin_grad(l2, out, o_weights, o_bias)
    l1.g = (l1 > 0).float() * l2.g
    lin_grad(inp, l1, h_weights, h_bias)

In [52]:
forward_and_backward(x_train, y_train)
o_weights.g

tensor([[  15.1010],
        [  39.7500],
        [ 940.9993],
        [ 771.4215],
        [1008.7953],
        [ 598.8926],
        [ 724.3143],
        [ 206.8452],
        [1034.5780],
        [  93.9648],
        [ 140.6766],
        [ 465.4284],
        [  44.4407],
        [  34.9705],
        [ 225.5081],
        [ 591.0358],
        [1565.5575],
        [ 808.0443],
        [1232.3121],
        [ 110.6135],
        [ 380.2662],
        [1460.4283],
        [1486.6355],
        [ 436.7134],
        [  48.7040],
        [1058.4683],
        [ 182.1762],
        [  93.7271],
        [ 845.0152],
        [ 337.5440],
        [ 301.0381],
        [  31.8464],
        [ 767.2654],
        [1053.3126],
        [   4.8145],
        [ 616.2936],
        [  28.1131],
        [  30.1059],
        [1250.2356],
        [ 953.7845],
        [1195.8608],
        [  91.5056],
        [ 238.6953],
        [ 517.3145],
        [ 625.9207],
        [ 906.5762],
        [ 255.8927],
        [ 461

## Refactor the model

In [40]:
# Reimplement our functions a callable classes, that calculate their own gradient


class Relu:
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.0)

        return self.out

    # Calculate our inputs gradient, chain with output grad
    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g


class Lin:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def __call__(self, inp):
        self.inp = inp
        self.out = self.inp @ self.weights + self.bias

        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.weights.t()
        self.weights.g = self.inp.t() @ self.out.g
        self.bias.g = self.out.g.sum(0)


class Mse:
    def __call__(self, inp, targets):
        self.inp = inp
        self.targets = targets

        self.out = (self.inp.squeeze() - self.targets).pow(2).mean()

        return self.out

    def backward(self):
        self.inp.g = 2.0 * (self.inp.squeeze() - self.targets).unsqueeze(-1) / self.targets.shape[0]

In [41]:
# Similar story for out model
class Model:
    def __init__(self, weights1, bias1, weights2, bias2):
        self.layers = [Lin(weights1, bias1), Relu(), Lin(weights2, bias2)]
        self.loss = Mse()

    def __call__(self, inp, targets):
        result = inp
        for layer in self.layers:
            result = layer(result)

        result = self.loss(result, targets)

        return result

    # Run backwards through our layers and calc the grad
    def backward(self):
        self.loss.backward()

        for layer in reversed(self.layers):
            layer.backward()

In [51]:
# Now we can build a model and run it
model = Model(h_weights, h_bias, o_weights, o_bias)
loss = model(x_train, y_train)
model.backward()

loss, o_weights.g

(tensor(6555.3013),
 tensor([[  15.1010],
         [  39.7500],
         [ 940.9993],
         [ 771.4215],
         [1008.7953],
         [ 598.8926],
         [ 724.3143],
         [ 206.8452],
         [1034.5780],
         [  93.9648],
         [ 140.6766],
         [ 465.4284],
         [  44.4407],
         [  34.9705],
         [ 225.5081],
         [ 591.0358],
         [1565.5575],
         [ 808.0443],
         [1232.3121],
         [ 110.6135],
         [ 380.2662],
         [1460.4283],
         [1486.6355],
         [ 436.7134],
         [  48.7040],
         [1058.4683],
         [ 182.1762],
         [  93.7271],
         [ 845.0152],
         [ 337.5440],
         [ 301.0381],
         [  31.8464],
         [ 767.2654],
         [1053.3126],
         [   4.8145],
         [ 616.2936],
         [  28.1131],
         [  30.1059],
         [1250.2356],
         [ 953.7845],
         [1195.8608],
         [  91.5056],
         [ 238.6953],
         [ 517.3145],
         [ 6

### Refactor with a Module

We can refactor with a Module base class.

In [53]:
class Module:
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self):
        raise Exception("Not Implemented")

    def bwd(self):
        raise exception("Not implemented")

    def backward(self):
        self.bwd(self.out, *self.args)

In [58]:
# Now our class implementations look like this


class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.0)

    def bwd(self, out, inp):
        inp.g = (inp > 0).float() * out.g


class Lin(Module):
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def forward(self, inp):
        return inp @ self.weights + self.bias

    def bwd(self, out, inp):
        inp.g = out.g @ self.weights.t()
        self.weights.g = inp.t() @ out.g
        self.bias.g = out.g.sum(0)


class Mse(Module):
    def forward(self, inp, targets):
        return (inp.squeeze() - targets).pow(2).mean()

    def bwd(self, out, inp, targets):
        inp.g = 2.0 * (inp.squeeze() - targets).unsqueeze(-1) / targets.shape[0]

In [60]:
# Everything still works and with less code
model = Model(h_weights, h_bias, o_weights, o_bias)
loss = model(x_train, y_train)
model.backward()

loss, o_weights.g

(tensor(6555.3013),
 tensor([[  15.1010],
         [  39.7500],
         [ 940.9993],
         [ 771.4215],
         [1008.7953],
         [ 598.8926],
         [ 724.3143],
         [ 206.8452],
         [1034.5780],
         [  93.9648],
         [ 140.6766],
         [ 465.4284],
         [  44.4407],
         [  34.9705],
         [ 225.5081],
         [ 591.0358],
         [1565.5575],
         [ 808.0443],
         [1232.3121],
         [ 110.6135],
         [ 380.2662],
         [1460.4283],
         [1486.6355],
         [ 436.7134],
         [  48.7040],
         [1058.4683],
         [ 182.1762],
         [  93.7271],
         [ 845.0152],
         [ 337.5440],
         [ 301.0381],
         [  31.8464],
         [ 767.2654],
         [1053.3126],
         [   4.8145],
         [ 616.2936],
         [  28.1131],
         [  30.1059],
         [1250.2356],
         [ 953.7845],
         [1195.8608],
         [  91.5056],
         [ 238.6953],
         [ 517.3145],
         [ 6

### Autograd

Obviously pytorch has done all of this for us (`nn.Module`).

In [62]:
from torch import nn
import torch.nn.functional as F

# We would implement out Lin class as this
# Note we dont have to implment that backward pass at all, just ask pytorch to with requires_grad


class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.weights = torch.randn(n_in, n_out).requires_grad_()
        self.bias = torch.zeros(n_out).requires_grad_()

    def forward(self, inp):
        return inp @ self.weights + self.bias


## We can reimplement our model using our Linear and borrwoing everything else from pytorch


class Model(nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super().__init__()
        self.layers = [Linear(n_in, n_hidden), nn.ReLU(), Linear(n_hidden, n_out)]

    def __call__(self, inp, targets):
        res = inp
        for layer in self.layers:
            res = layer(res)

        res = F.mse_loss(res, targets[:, None])

        return res

In [73]:
# Everything still works and with even less code
model = Model(n_pixels, n_hidden, 1)
loss = model(x_train, y_train)
loss.backward()

loss, model.layers[2].weights.grad

(tensor(1223.1824, grad_fn=<MseLossBackward0>),
 tensor([[ 4.3840e+01],
         [ 1.1604e+02],
         [-5.3228e+01],
         [ 1.2646e+02],
         [ 7.8353e+01],
         [ 7.8051e+01],
         [ 1.5563e+01],
         [ 3.5694e+01],
         [ 1.4275e+02],
         [ 1.6304e+01],
         [ 1.4957e+02],
         [ 3.2792e+01],
         [ 7.8994e+01],
         [ 4.8192e+01],
         [ 1.3647e+02],
         [ 1.4502e+02],
         [ 9.2334e+01],
         [ 7.9152e+01],
         [ 1.1404e+01],
         [ 4.3579e+01],
         [ 1.1250e+02],
         [ 6.5240e+00],
         [ 1.8020e+02],
         [ 1.7331e+02],
         [ 1.1430e+02],
         [ 3.1277e+02],
         [ 5.6202e+01],
         [ 3.5445e+02],
         [ 1.8891e+02],
         [-1.8835e+01],
         [ 1.4538e+02],
         [ 1.4102e+01],
         [ 2.5614e+01],
         [ 1.4344e-01],
         [ 4.2751e+02],
         [ 6.4488e+01],
         [ 3.9571e+01],
         [ 8.8182e+01],
         [ 4.8206e+02],
         [-4.347